In [1]:
import pandas as pd
import requests
import numpy as np
import re
from bs4 import BeautifulSoup as bs
from lxml import etree, html
from IPython.display import display, HTML

In [2]:
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 30)

In [3]:
url='ссылка на csv на гугл диске'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
vacancies = pd.read_csv(url)

#### Сделаем отдельный столбец с текстом, полученным из html разметки (для последующего поиска хард скиллов в этом тексте) 

In [4]:
vacancies['raw_text'] = ''
for text_num in range(vacancies.shape[0]):
    html = vacancies['html'][text_num]
    soup = bs(html)
    vacancies['raw_text'][text_num] = soup.text.strip().lower().replace('\n', '')

C:\Users\Boris Bobina\AppData\Local\Temp\ipykernel_8552\2912845910.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacancies['raw_text'][text_num] = soup.text.strip().lower().replace('\n', '')


#### Наименование вакансии

In [5]:
vacancies['name'] = vacancies['html'].apply(lambda x: bs(x).find('h2').text.strip())

Отфильтруем датафрейм и оставим в нем только вакансии, которые содержат слово "analyst"

In [6]:
vacancies = vacancies[vacancies['name'].str.contains('analyst|Analista de datos|Analityk danych|ANALYSTE DES DONNEES', case=False)]

#### Город

In [7]:
vacancies['city'] = vacancies['html'].apply(lambda x: bs(x).find('span', class_ = 'jobs-unified-top-card__bullet').text.strip().split(',')[0].strip() if len(bs(x).find('span', class_ = 'jobs-unified-top-card__bullet').text.strip().split(',')) >=2  else np.nan)
vacancies['city'].isnull().mean()

0.1072463768115942

#### Страна

In [8]:
vacancies['country'] = vacancies['html'].apply(lambda x: bs(x).find('span', class_ = 'jobs-unified-top-card__bullet').text.strip().split(',')[-1].strip() if len(bs(x).find('span', class_ = 'jobs-unified-top-card__bullet').text.strip().split(',')) == 3 else np.nan)
vacancies['country'].isnull().mean()

0.17101449275362318

Пропсков в столбце с городами меньше, чем пропусков в столбце с странами. Можем за счет информации о городах дополнить информацию о странах. В этом нам поможет GptChat: напишет словарь в формате {'город': 'страна'}

In [9]:
city_dict = {
    'Basel': 'Switzerland',
    'Coventry': 'United Kingdom',
    'South Molton': 'United Kingdom',
    'Lugano': 'Switzerland',
    'Southampton': 'United Kingdom',
    'Leeds': 'United Kingdom',
    'Nuneaton': 'United Kingdom',
    'Paris': 'France',
    'Cambridge': 'United Kingdom',
    'West Midlands': 'United Kingdom',
    'Schiphol': 'Netherlands',
    'Chester': 'United Kingdom',
    'Craven Arms': 'United Kingdom',
    'Dublin': 'Ireland',
    'Belfast': 'United Kingdom',
    'Sunderland': 'United Kingdom',
    'Montévrain': 'France',
    'Bristol': 'United Kingdom',
    'Solihull': 'United Kingdom',
    'Blackpool': 'United Kingdom',
    'Cracow': 'Poland',
    'Dijon': 'France',
    'Alsónémedi': 'Hungary',
    'Manchester': 'United Kingdom',
    'Elliniko-Argyroupoli': 'Greece',
    'Umeå': 'Sweden',
    'North Holland': 'Netherlands',
    'Vilnius': 'Lithuania',
    'Durham': 'United Kingdom',
    'Oudenaarde': 'Belgium',
    'Milan': 'Italy',
    'Stockholm County': 'Sweden',
    'Luxembourg': 'Luxembourg',
    'Roubaix': 'France',
    'Munich': 'Germany',
    'Zaventem': 'Belgium',
    'West Malling': 'United Kingdom',
    'Lille': 'France',
    'Egham': 'United Kingdom',
    'Karlstad': 'Sweden',
    'Madrid': 'Spain',
    'Barcelona': 'Spain',
    'Oxford': 'United Kingdom',
    'Brussels': 'Belgium',
    'Taibon': 'Italy',
    'Epsom': 'United Kingdom',
    'Amsterdam': 'Netherlands',
    'Spinea': 'Italy',
    'Brindisi': 'Italy',
    'Boulogne-Billancourt': 'France',
    'Wolfsburg': 'Germany',
    'Nantes': 'France',
    'Derby': 'United Kingdom',
    'Lund': 'Sweden',
    'Garwolin': 'Poland',
    'Stockholm': 'Sweden',
    'Rome': 'Italy',
    'Massy': 'France',
    'Prague': 'Czech Republic',
    'Middlesbrough': 'United Kingdom',
    'Viladecans': 'Spain',
    'Eindhoven': 'Netherlands',
    'Warsaw': 'Poland',
    'Budapest': 'Hungary',
    'London': 'United Kingdom',
    'Amersfoort': 'Netherlands',
    'Hamburg': 'Germany',
    'The Hague': 'Netherlands',
    'Chappes': 'France',
    'Sintra': 'Portugal',
    'Riga': 'Latvia',
    'Coimbra': 'Portugal',
    'Tartu': 'Estonia',
    'Île-de-France': 'France',
    'Issy-les-Moulineaux': 'France',
    'Hawthorn': 'Australia',
    'Lyon': 'France',
    'Valletta': 'Malta',
    'Stuttgart': 'Germany',
    'Galway': 'Ireland',
    'Berlin': 'Germany',
    'Olèrdola': 'Spain',
    'Neuilly-sur-Seine': 'France',
    'Wasserburg (Bodensee)': 'Germany',
    'Gdynia': 'Poland',
    'Bordeaux': 'France',
    'Rouen': 'France',
    'Mechelen': 'Belgium',
    'Milton Keynes': 'United Kingdom',
    'Bari': 'Italy',
    'Vienna': 'Austria',
    'Newcastle upon Tyne': 'United Kingdom',
    'Cerdanyola del Vallès': 'Spain',
    'Naples': 'Italy',
    'Nova Milanese': 'Italy',
    'Baierbrunn': 'Germany',
    'Parma': 'Italy',
    'Lindau': 'Germany',
    'Bergen op Zoom': 'Netherlands',
    'Gdańsk': 'Poland',
    'Bodelshausen': 'Germany',
    'Marlow': 'United Kingdom',
    'Brussels Region': 'Belgium',
    'La Défense': 'France',
    'Lisbon': 'Portugal',
    'Porto': 'Portugal',
    'The Randstad': 'Netherlands',
    'Neuss': 'Germany',
    'Ghent': 'Belgium',
    'San Ġiljan': 'Malta',
    'Cologne': 'Germany',
    'Rijswijk': 'Netherlands',
    'Tessenderlo': 'Belgium',
    'St.-Ouen': 'France',
    'Buckinghamshire': 'United Kingdom',
    'Bologna': 'Italy',
    'Hasselt': 'Belgium',
    'Bath': 'United Kingdom',
    'Dresden': 'Germany',
    'Wiesbaden': 'Germany',
    'Utrecht': 'Netherlands',
    'Mülheim an der Ruhr': 'Germany',
    'Zwaagdijk-Oost': 'Netherlands',
    'Heerlen': 'Netherlands',
    'Hatfield': 'United Kingdom',
    'Harlestone': 'United Kingdom',
    'Nanterre': 'France',
    'Hook of Holland': 'Netherlands',
    'Oslo': 'Norway',
    'Szczecin': 'Poland',
    'Villes': 'France',
    'Doncaster': 'United Kingdom',
    'County Dublin': 'Ireland',
    'Villeneuve-d’Ascq': 'France',
    'Toulouse': 'France',
    'Meudon': 'France',
    'Padua': 'Italy',
    'Rotterdam': 'Netherlands',
    'Levallois-Perret': 'France',
    'Copenhagen': 'Denmark',
    'Espoo': 'Finland',
    'Karlsruhe': 'Germany',
    'Limburg': 'Belgium',
    'Schwäbisch Gmünd': 'Germany',
    'Sundsvall': 'Sweden',
    'Neckarsulm': 'Germany',
    'Düsseldorf': 'Germany',
    'Gillingham': 'United Kingdom',
    'Bucharest': 'Romania',
    'Emmen': 'Netherlands',
    'Guissona': 'Spain',
    'Hoofddorp': 'Netherlands',
    'Weinheim': 'Germany',
    'West Flanders': 'Belgium',
    'BustoArsizio':'Italy',
    'Lisboa':'Portugal',
    'Jena':'Germany',
    'Worksop':'UnitedKingdom',
    'Vilvoorde':'Belgium',
    'BergischGladbach':'Germany',
    'Drammen':'Norway',
    'Frankfurt':'Germany',
    'Athens':'Greece',
    'Zeist':'Netherlands',
    'Groningen':'Netherlands',
    'Louvain-la-Neuve':'Belgium',
    'Mannheim':'Germany',
    'Łódź':'Poland',
    'St.-Denis':'France',
    'Grenzach-Wyhlen':'Germany',
    'Essen':'Germany',
    'Leipzig':'Germany',
    'Iráklion':'Greece',
    "Vapriod'Adda":'Italy',
    'Gloucestershire':'UnitedKingdom',
    'FrankfurtamMain':'Germany',
    'Ahlen':'Germany',
    'Sofia':'Bulgaria',
    'Cesate':'Italy',
    'Binasco':'Italy',
    'BellinzagoLombardo':'Italy',
    'CernuscosulNaviglio':'Italy',
    'ManchesterArea':'UnitedKingdom',
    'Colturano':'Italy',
    'Truccazzano':'Italy',
    'Inzago':'Italy',
    'Pistoia':'Italy',
    'Canegrate':'Italy',
    'Lombardy':'Italy',
    'Coburg':'Germany',
    'SanZenonealLambro':'Italy',
    "Cassinade'Pecchi":'Italy',
    'Monaco':'Monaco',
    'Höchberg':'Germany',
    'Palermo':'Italy',
    'Treviso':'Italy',
    'Thessaloniki':'Greece',
    'Courbevoie':'France',
    'Doesburg':'Netherlands',
    'Tussenhausen':'Germany',
    'Arluno':'Italy',
    'Hückelhoven':'Germany',
    'Eisenach':'Germany',
    'Hereford':'UnitedKingdom',
    'Erstein':'France',
    'Szombathely':'Hungary',
    'Ronneby':'Sweden',
    'Elmshorn':'Germany',
    'Pioltello':'Italy',
    'Bremen':'Germany',
    'Lidköping':'Sweden',
    'Ede':'Netherlands',
    'Vannes':'France',
    'Vittuone':'Italy',
    'Faenza':'Italy',
    'Tallinn':'Estonia',
    'Farum':'Denmark',
    'ZianodiFiemme':'Italy',
    'Rethymno':'Greece',
    'Braine-l’Alleud':'Belgium',
    'VilaNovadeCerveira':'Portugal',
    'Campodarsego':'Italy',
    'Bonn':'Germany',
    'Hertfordshire':'UnitedKingdom',
    'Lelystad':'Netherlands',
    'Marsa':'Malta',
    'Elsbethen':'Austria',
    'Augusta':'Italy',
    'Ottignies-Louvain-la-Neuve': 'Belgium',
    'Fuschl am See': 'Austria',
    'Solna': 'Sweden',
    'Wrocław': 'Poland',
    'Salzburg': 'Austria',
    'Senago': 'Italy',
    'Åre': 'Sweden',
    'Bilbao-Bilbo': 'Spain',
    'Bois-Colombes': 'France',
    'Vitoria-Gasteiz': 'Spain',
    'Cusano Milanino': 'Italy',
    'Dresano': 'Italy',
    'Clichy': 'France',
    'Douai': 'France',
    "'s-Hertogenbosch": 'Netherlands',
    'Kaiserslautern': 'Germany',
    'Zoetermeer': 'Netherlands',
    'Cagliari': 'Italy',
    'Moulineaux': 'France',
    'Zagreb': 'Croatia',
    'Flemish Region': 'Belgium',
    'Zwevezele': 'Belgium',
    'Košice': 'Slovakia',
    'Calvignasco': 'Italy',
    'Telde': 'Spain',
    'Ronda': 'Spain',
    'Chevigny-Saint-Sauveur': 'France',
    'Gessate': 'Italy',
    'Arrasate / Mondragón': 'Spain',
    'Rozzano': 'Italy',
    'Vanzago': 'Italy',
    'Esch-Sur-Alzette': 'Luxembourg',
    'Gattinara': 'Italy',
    'Danderyd': 'Sweden',
    'Sesto San Giovanni': 'Italy',
    'Turin': 'Italy',
    'Savona': 'Italy',
    'Cornaredo': 'Italy',
    'Torremolinos': 'Spain',
    'Lucerne': 'Switzerland',
    'Monza': 'Italy',
    'Carpi': 'Italy',
    'County Cork': 'Ireland',
    'Poissy': 'France',
    'Kontich': 'Belgium',
    'Frosinone': 'Italy',
    'Amstelveen': 'Netherlands',
    'San Donato Milanese': 'Italy',
    'Valladolid': 'Spain',
    'Suresnes': 'France',
    'Laudio / Llodio': 'Spain',
    'Holzwickede': 'Germany',
    'Palaión Fáliron': 'Greece',
    'Lacchiarella': 'Italy',
    "Sant'Elpidio a Mare": 'Italy',
    'Terranuova Bracciolini': 'Italy',
    'Gennevilliers': 'France',
    'Sparta': 'Greece',
    'Brunswick': 'Germany',
    'Corsico': 'Italy',
    'Aalborg': 'Denmark',
    'Florence': 'Italy',
    'Chaucer': 'United Kingdom',
    'Dobříš': 'Czech Republic',
    'Heidelberg': 'Germany',
    'Liguria': 'Italy',
    'Cusago': 'Italy',
    'Oeiras': 'Portugal',
    'Leinfelden-Echterdingen': 'Germany',
    'Timişoara': 'Romania',
    'Nuremberg': 'Germany',
    'Edinburgh': 'United Kingdom',
    'Bertrange': 'Luxembourg',
    'Piraeus': 'Greece',
    'Arnhem': 'Netherlands',
    'Sliema': 'Malta',
    'Aix-en-Provence': 'France',
    'Gießen': 'Germany',
    'Debrecen': 'Hungary',
    'Bergamo': 'Italy',
    'Ulm': 'Germany',
    'Grünheide': 'Germany',
    'Leiden': 'Netherlands',
    'Boadilla del Monte': 'Spain',
    'Bracknell': 'United Kingdom',
    'Venlo': 'Netherlands',
    'Münster': 'Germany',
    'Orosháza': 'Hungary',
    'Osnabrück': 'Germany',
    'Vitry-sur-Seine': 'France',
    'Mönchengladbach': 'Germany',
    'Carpiano': 'Italy',
    'Rosate': 'Italy',
    'Gaydon': 'United Kingdom',
    'Pordenone': 'Italy',
    'Kosice': 'Slovakia',
    'Charenton-le-Pont': 'France',
    'Liège': 'Belgium',
    'Raamsdonksveer': 'Netherlands',
    'Matera': 'Italy',
    'Jonkoping County': 'Sweden',
    'Croix': 'France',
    'Helsinki': 'Finland',
    'Luleå': 'Sweden',
    'Genoa': 'Italy',
    'Bernay': 'France',
    'Helsingborg': 'Sweden',
    'Fontenay-sous-Bois': 'France',
    'Vila do Conde': 'Portugal',
    'Mainz': 'Germany',
    'Bollate': 'Italy',
    'Pieve Emanuele': 'Italy',
    'Guyancourt': 'France',
    'Sant Just Desvern': 'Spain',
    'Nice': 'France',
    'San Colombano al Lambro': 'Italy',
    'Dublin City': 'Ireland',
    'Basingstoke': 'United Kingdom',
    'Sundbyberg': 'Sweden',
    'Albuzzano': 'Italy',
    'Arconate': 'Italy',
    'Birkirkara': 'Malta',
    'St-Malo': 'France',
    'Sligo': 'Ireland',
    'Crolles': 'France',
    'Trezzano Rosa': 'Italy',
    'Plaisir': 'France',
    'Imola': 'Italy',
    'Bègles': 'France',
    'Novate Milanese': 'Italy',
    'Waddinxveen': 'Netherlands',
    'Palaiseau': 'France',
    'Szentendre': 'Hungary',
    'Moravia-Silesia': 'Czech Republic',
    "Cassano d'Adda": 'Italy',
    'Puteaux': 'France',
    'Székesfehérvár': 'Hungary',
    'Cork': 'Ireland',
    "Granarolo dell'Emilia": 'Italy',
    'Malmö': 'Sweden',
    'Sedriano': 'Italy',
    'Nitra Region': 'Slovakia',
    'Paterna': 'Spain',
    'Ancona': 'Italy',
    'Vanzaghello': 'Italy',
    'Recanati': 'Italy'
}

In [10]:
def fill_missing_countries(city):
    return city_dict.get(city, None)

vacancies['country'] = vacancies['city'].apply(fill_missing_countries)
vacancies['country'].isna().mean()

0.13478260869565217

Удалось уменьшить количество пропусков в столбце стран

#### Тип занятости

In [11]:
vacancies['employment_type'] = vacancies['html'].apply(lambda x: bs(x).find('span', class_='jobs-unified-top-card__workplace-type').text if bs(x).find('span', class_='jobs-unified-top-card__workplace-type') else np.nan)

#### Компания

In [12]:
vacancies['company_name'] = vacancies['html'].apply(lambda x: bs(x).find('span', class_='jobs-unified-top-card__company-name').text.strip())

#### Размер компании

In [13]:
vacancies['employee_qty'] = vacancies['html'].apply(lambda x: bs(x).find_all('li', class_='jobs-unified-top-card__job-insight')[1].text.strip().split(' · ')[0] if bs(x).find_all('li', class_='jobs-unified-top-card__job-insight') else np.nan)

#### Сфера деятельности компании

In [14]:
vacancies['company_scope'] = vacancies['html'].apply(lambda x: bs(x).find_all('li', class_='jobs-unified-top-card__job-insight')[1].text.strip().split(' · ')[-1] if bs(x).find_all('li', class_='jobs-unified-top-card__job-insight') else np.nan)

#### Требуемые хард скилы

Список хард скиллов аналитика данных

In [15]:
vacancies['required_skills'] = vacancies['html'].apply(lambda x: bs(x).find('span', class_='visually-hidden').text.strip() if bs(x).find('span', class_='visually-hidden') else np.nan)

In [16]:
new_skill_list = []
pattern = r'Skills:\s(.+),\s\+'
vacancies['required_skills'] = vacancies['required_skills'].str.extract(pattern)
skill_list = list(vacancies['required_skills'].unique())
for skill in skill_list:
    try:
        for i in range(len(skill)):
            new_skill_list.append(skill.split(', ')[i].lower())
    except:
        pass
print(new_skill_list)

['анализ данных', 'визуализация данных', 'панель управления', 'анализ данных', 'data analytics', 'компьютерная наука', 'ssrs', 'услуги анализа', 'sql', 'анализ бизнес-данных', 'аналитические навыки', 'powerpoint', 'анализ данных', 'аналитические навыки', 'etl tools', 'английский язык', 'статистическое моделирование', 'моделирование данных', 'python', 'sql', 'визуализация данных', 'tableau', 'google analytics', 'анализ данных', 'sql', 'анализ данных', 'google data studio', 'looker (software)', 'business analysis', 'microsoft excel', 'командная работа', 'аналитические навыки', 'tableau', 'немецкий язык', 'анализ данных', 'английский язык', 'ad hoc analysis', 'анализ данных', 'английский язык', 'анализ данных', 'microsoft office', 'аналитические навыки', 'python', 'управление проектами', 'data analytics', 'visualization', 'английский язык', 'французский язык', 'python', 'scala', 'экономика', 'математика', 'английский язык', 'microsoft excel', 'аналитические навыки', 'sql', 'межличностное 

In [17]:
skills = set(new_skill_list)

Ищем скиллы в тексте

In [18]:
def skills_finder(row, skill_list_2=skills):

    matched_skills_list=[]
    row = str(row)

    for skill in skill_list_2:
        if skill == 'c++':
            if re.search('\Wc\+\+\W', row.lower()):
                matched_skills_list.append(skill)  

        else:
            pattern = (
            r'(\b|\W)'
            + re.escape(skill)
            + r'(\b|\W)'
            +'|'
            + r'(\b|\W)'
            +re.escape(skill.replace(' ', ''))
            + r'(\b|\W)'
            )
          
            if re.search(pattern, row.lower()):
                matched_skills_list.append(skill)

    return matched_skills_list

In [19]:
vacancies['required_skills'] = vacancies['raw_text'].apply(skills_finder)

#### Количество кандидатов на вакансию

In [20]:
vacancies['applicants'] = vacancies['html'].apply(lambda x: bs(x).find('span', class_='jobs-unified-top-card__applicant-count').text if bs(x).find('span', class_='jobs-unified-top-card__applicant-count') else None)

In [21]:
vacancies['applicants'] = vacancies['applicants'].str.extract('(\d+)', expand=False)
vacancies['applicants'] = pd.to_numeric(vacancies['applicants'], errors='coerce')

#### Дата публикации вакансии

In [22]:
rep_dict = {'weeks': 'week', 'days': 'day', ' ago': ''}
vacancies['time_delta'] = vacancies['html'].apply(lambda x: bs(x).find('span', class_='jobs-unified-top-card__posted-date').text.strip())
vacancies['time_delta'] = vacancies['time_delta'].replace(rep_dict, regex=True)

In [23]:
vacancies['time_delta']

0      1 week
1      1 week
2      1 week
3      1 week
4      2 week
        ...  
990    2 week
991    1 week
994    2 week
995     1 day
997    2 week
Name: time_delta, Length: 690, dtype: object

In [24]:
def convert_to_timedelta(duration):
    num, unit = duration.split()
    if unit == 'week':
        return pd.Timedelta(weeks=int(num))
    elif unit == 'day':
        return pd.Timedelta(days=int(num))
    else:
        pass

In [25]:
base_date = pd.to_datetime('2023-05-23')

timedeltas = vacancies['time_delta'].apply(convert_to_timedelta)

vacancies['published_at'] = base_date + timedeltas
vacancies[['time_delta', 'published_at']]

,time_delta,published_at
0,1 week,2023-05-30
1,1 week,2023-05-30
2,1 week,2023-05-30
3,1 week,2023-05-30
4,2 week,2023-06-06
...,...,...
990,2 week,2023-06-06
991,1 week,2023-05-30
994,2 week,2023-06-06
995,1 day,2023-05-24


In [26]:
vacancies = vacancies[['name', 'city', 'country', 'employment_type', 'company_name', 'employee_qty', 'company_scope', 'required_skills', 'applicants', 'published_at']]

In [27]:
vacancies.head(20)

,name,city,country,employment_type,company_name,employee_qty,company_scope,required_skills,applicants,published_at
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,"[spss, data analytics, personal data, sas, sap, sql]",47.0,2023-05-30
1,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,NaN,NaN,[],NaN,2023-05-30
2,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,NaN,NaN,[data analytics],NaN,2023-05-30
3,Data Analyst (Space & Planning),South Molton,United Kingdom,On-site,Mole Valley Farmers,NaN,NaN,[],NaN,2023-05-30
4,Data Analyst,Lugano,Switzerland,On-site,FORFIRM,NaN,NaN,"[visualization, etl tools, python, etl, sql]",NaN,2023-06-06
5,Data Analyst - Logistics,Southampton,United Kingdom,On-site,"Butler, Bridge & May",NaN,NaN,[sap],NaN,2023-05-29
6,Data Analyst,Leeds,United Kingdom,On-site,Maria Mallaband Care Group Ltd,NaN,NaN,[microsoft excel],NaN,2023-06-13
7,Data Analyst,Nuneaton,United Kingdom,Hybrid,Kelly Group,NaN,NaN,"[interpreting data, microsoft office, attention to detail]",NaN,2023-05-25
8,Data Analyst,Paris,France,On-site,eXalt,"501-1,000 employees",IT Services and IT Consulting,"[qlikview, tableau, визуализация данных, sql, esprit, анализ данных]",140.0,2023-06-06
9,Data Analyst - Hybrid Working,Cambridge,United Kingdom,On-site,Blue Arrow,NaN,NaN,[interpreting data],NaN,2023-06-06


In [28]:
vacancies.to_csv('vacancies.csv', index=False)

In [29]:
skills = vacancies.explode('required_skills')[['required_skills']]

In [30]:
skills.value_counts().head(20)

required_skills     
sql                     242
python                  193
r                       142
data analytics          135
tableau                 120
powerpoint               75
аналитические навыки     70
visualization            67
английский язык          61
визуализация данных      60
commerce                 56
attention to detail      51
анализ данных            51
business analysis        49
sap                      47
crm                      47
коммуникация             45
microsoft office         42
vba                      42
cultural affairs         39
dtype: int64

In [31]:
skills.to_csv('skills.csv', index=False)